# Environment Prepare

In [1]:
# !pip install keras_bert

In [2]:
from driver_amount import addh

[Locale] Using address head: /mnt/d/My Drive


In [3]:
# TODO Need a method about how to check git auto

# !git clone https://github.com/supplient/bachelor_design.git
# import os
# os.chdir("bachelor_design")
# !pwd

In [4]:
# !git checkout equal_realize
# !git pull

# Data Prepare

Origin Data

In [5]:
from preprocess import load_file
import config
import json

In [6]:
char_seqs, tag_seqs = load_file(addh + config.DATA_PATH)
equal_seqs = None
with open(addh + config.EQUAL_DATA_PATH, "r") as fd:
    equal_seqs = json.load(fd)
origin_seqs = char_seqs[:len(equal_seqs)]

# Param Load

In [7]:
import json
params = None
# TODO the better way to do this is 
#    check whether the file exists first,
#    then create it and set default params when it does not exist.
params = {}
with open(addh + config.EQUAL_PARAM_PATH, "r") as fd:
    params = json.load(fd)
    
sif_alpha = params["sif_alpha"]
cos_theta = params["cos_theta"]

dist_theta = cos_theta

# Embedding

Since our embedding is designed for batch work, it should be better if we combine origin_seqs and equal_seqs.

In [8]:
origin_num = len(origin_seqs)
all_char_seqs = []
all_char_seqs.extend(origin_seqs)
all_char_seqs.extend(equal_seqs)

In [9]:
from char_emb import CharEmbedder
char_embedder = CharEmbedder()
all_char_emb_seqs = char_embedder.embed(all_char_seqs)

Using TensorFlow backend.
/home/supplient/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/supplient/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/supplient/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/supplient/.local/lib/python3.6/site-packages/tenso

In [10]:
from SIF import SIF
sif = SIF(sif_alpha)
all_sen_vecs = sif.compose(all_char_seqs, all_char_emb_seqs)

# Compare & Fine-tune params

In [11]:
origin_sen_vecs = all_sen_vecs[:origin_num]
equal_sen_vecs = all_sen_vecs[origin_num:]
if len(origin_sen_vecs) != len(equal_sen_vecs):
    raise Exception("Length should be the same.")

Set some train params

In [21]:
epoch = 10000
delta = 0.001
min_delta = 10**(-10)

Use two experiments:
* Check whether origin and equal are similiar
* Check whether origins are different

In [16]:
from dist_cal import DistCal
dist_cal = DistCal()

In [25]:
from tqdm import tnrange

similiar_rate = 0
different_rate = 0
last_delta = 0
with tnrange(epoch) as epoch_tqdm:
    for epoch_count in epoch_tqdm:
        similiar_count = 0
        similiar_total = len(origin_sen_vecs)
        for origin_sen, equal_sen in zip(origin_sen_vecs, equal_sen_vecs):
            dist = dist_cal.cal(origin_sen, equal_sen)
            if dist < dist_theta:
                similiar_count += 1

        N = len(origin_sen_vecs)
        different_count = 0
        different_total = N * (N -1)/2
        for i in range(N-1):
            for j in range(i+1, N):
                dist = dist_cal.cal(origin_sen_vecs[i], origin_sen_vecs[j])
                if dist > dist_theta:
                    different_count += 1

        similiar_rate = similiar_count / similiar_total
        different_rate = different_count / different_total

        now_delta = 0
        if similiar_rate > different_rate:
            now_delta = -delta
        elif similiar_rate < different_rate:
            now_delta = delta
        else:
            now_delta = -last_delta
            
        if now_delta == -last_delta:
            delta /= 10
            now_delta /= 10
        dist_theta += now_delta
        last_delta = now_delta
            
        if dist_theta <= 0:
            raise Exception("dist_theta reach 0")
        if dist_theta >= 1:
            raise Exception("dist_theta reach 1")
            
        epoch_tqdm.set_description("Epoch %i" % epoch_count)
        epoch_tqdm.set_postfix(
            similiar_rate=similiar_rate, 
            different_rate=different_rate,
            dist_theta=dist_theta,
            delta=delta
        )
            
        if delta <= min_delta:
            epoch_tqdm.write("Delta reach " + str(min_delta) + ", which is enough")
            break

/home/supplient/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  


Delta reach 1e-17, which is enough



# Save params

In [24]:
cos_theta = dist_theta

params["sif_alpha"] = sif_alpha
params["cos_theta"] = cos_theta

with open(addh + config.EQUAL_PARAM_PATH, "w") as fd:
    json.dump(params, fd, indent=4)